In [ ]:
repo_dir = "Repos"   # Set this to be where your github repos are located.
%load_ext autoreload
%autoreload 2

# Update the load path so python can find modules for the model
import sys
from pathlib import Path
sys.path.insert(0, str(Path.home() / repo_dir / "eye-ai-ml"))

In [ ]:
# Prerequisites

import json
import os
from eye_ai.eye_ai import EyeAI
import pandas as pd
from pathlib import Path, PurePath
import logging
# import torch

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

In [ ]:
from deriva.core.utils.globus_auth_utils import GlobusNativeLogin
catalog_id = "eye-ai" #@param
host = 'www.eye-ai.org'


gnl = GlobusNativeLogin(host=host)
if gnl.is_logged_in([host]):
    print("You are already logged in.")
else:
    gnl.login([host], no_local_server=True, no_browser=True, refresh_tokens=True, update_bdbag_keychain=True)
    print("Login Successful")

In [ ]:
# Variables to configure the rest of the notebook.

cache_dir = '/data'        # Directory in which to cache materialized BDBags for datasets
working_dir = '/data'    # Directory in which to place output files for later upload.

configuration_rid="2-BDA8" # rid


In [ ]:
EA = EyeAI(hostname = host, catalog_id = catalog_id, cache_dir= cache_dir, working_dir=working_dir)

In [ ]:
# @title Initiate an Execution
configuration_records = EA.execution_init(configuration_rid=configuration_rid)
configuration_records.model_dump()

In [ ]:
# View data

subject = pd.read_csv(configuration_records.bag_paths[0]/'data/Subject.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
subject

observation = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
observation

clinic = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinical_Records.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
clinic

observation_clinic_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Observation_Clinic_Asso.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
observation_clinic_asso # association table between observation table and clinic record table

icd10 = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD10.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
icd10

icd10_asso = pd.read_csv(configuration_records.bag_paths[0]/'data/Clinic_ICD_Asso.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
icd10_asso # association table between clinic record table and ICD10 code

report = pd.read_csv(configuration_records.bag_paths[0]/'data/Report.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
report

RNFL_OCR = pd.read_csv(configuration_records.bag_paths[0]/'data/RNFL_OCR.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
RNFL_OCR

HVF_OCR = pd.read_csv(configuration_records.bag_paths[0]/'data/HVF_OCR.csv').drop(columns=['RCT', 'RMT', 'RCB', 'RMB'])
HVF_OCR


In [ ]:
import pandas as pd
# Merge Subject with Observation
subject = subject.drop(columns=['hba1c', 'glaucoma_hx', 'visual_acuity_right',
       'visual_acuity_left', 'date_of_encounter', 'provider', 'reviewed_date',
       'dr_level', 'consult_id', 'date_created', 'assessment_and_recommendation', 'additional_comments',
       'return_time_frame', 'referral_status_time_frame', 'Subject_hypertension',
       'Subject_insulin_dependent', 'Subject_pregnant', 'Subject_cataract',
       'Subject_maculopathy', 'Subject_other', 'Subject_image_quality', 'site_mrn', 'Consultant'])

subject_observation = pd.merge(subject, observation, left_on='RID', right_on='Subject', how='left',
                               suffixes=('_Subject', '_Observation')).drop(columns=['Subject'])
subject_observation


In [ ]:
# Merge with Observation_Clinical_Association
subject_obs_clinic = pd.merge(subject_observation, 
                              observation_clinic_asso, 
                              left_on= 'RID_Observation', 
                              right_on='Observation', 
                              suffixes=('_subject_observation_df', "_observation_clinic_asso_df"),
                              how='left').drop(columns=['RID', 'Observation'])


In [ ]:
subject_obs_clinic

In [ ]:
# Merge with Clinical_Data
subject_obs_clinic_data = pd.merge(subject_obs_clinic, 
                                   clinic,
                                   left_on='Clinical_Records', 
                                   right_on='RID',
                                   suffixes=("_Subject_Obs_Clinic", "_Clinic"),
                                   how='left').drop(columns = ['Clinical_Records']).rename(columns={'RID':'RID_Clinic'})


In [ ]:
subject_obs_clinic_data


In [ ]:
subject_obs_clinic_data[['IOP', 'CDR']]

In [ ]:
count_clinic = pd.DataFrame(subject_obs_clinic_data.groupby(['RID_Observation', 'Powerform_Laterality'])['RID_Clinic'].count())
count_clinic


In [ ]:
count_clinic[count_clinic['RID_Clinic'] != 1]

In [ ]:
HVF_OCR

In [ ]:
# create subject, observation, report, hvf


subject_observation_for_HVF = subject_observation.copy()

subject_observation_for_HVF


In [ ]:
# Merge with Report
subject_observation_for_HVF_report = pd.merge(subject_observation_for_HVF, report, left_on='RID_Observation', right_on='Observation', suffixes=("subject_observation_for_HVF", "report"), how='left')


subject_observation_for_HVF_report


In [ ]:
# Merge with HVF_OCR
subject_observation_for_HVF_report_hvf = pd.merge(subject_observation_for_HVF_report, HVF_OCR, left_on='RID', right_on='Report', suffixes=("_subject_observation_for_HVF_report", "_HVF_OCR"), how='left')

subject_observation_for_HVF_report_hvf

In [ ]:

# create subject, observation, report, RNFL


subject_observation_for_RNFL = subject_observation.copy()

subject_observation_for_RNFL


In [ ]:
# Merge with Report
subject_observation_for_RNFL_report = pd.merge(subject_observation_for_RNFL, report, left_on='RID_Observation', right_on='Observation', suffixes=("_subject_observation_for_RNFL", "_report"), how='left')


subject_observation_for_RNFL_report


In [ ]:
# Merge with RNFL_OCR
subject_observation_for_RNFL_report_rnfl = pd.merge(subject_observation_for_RNFL_report, RNFL_OCR, left_on='RID', right_on='Report', suffixes=("_subject_observation_for_RNFL_report", "_RNFL_OCR"), how='left')

subject_observation_for_RNFL_report_rnfl



In [ ]:

# do create subject, observation, image

subject_observation_for_image = subject_observation.copy()

subject_observation_for_image

In [ ]:
pd.read_csv(configuration_records.bag_paths[0]/'data/Image.csv') 

In [ ]:
icd10_asso

In [ ]:
# ICD10_Eye find H 40.0 .1 and .2 find rid and do group by clinical record and count number from either 3 of them...

# convert 40. 0 \\\\



# we need to do the mapping icd rid with icd 10 code (3 digits code in the table)... map this with icd 10 code 1 digit



# Join icd10 and icd 10 asso, then group by 3 categories and assign new categories, and push the code... 
# see group by clinical record to see if more than 1 of the categories belong to the same row if duplicates or missing 


In [ ]:
icd10

In [ ]:

# Mapping ICD10 codes to categories
icd_mapping = {
    'H40.0*': 'GS', 
    'H40.1*': 'POAG', 
    'H40.2*': 'PACG'
}

# Function to apply the mapping
def map_icd_to_category(icd_code):
    for key, value in icd_mapping.items():
        if icd_code.startswith(key[:-1]):
            return value
    return 'Other'

# Apply the mapping
icd10['Category'] = icd10['ICD10'].apply(map_icd_to_category)

# Joining the tables
combined = pd.merge(icd10_asso, icd10, left_on='ICD10_Eye', right_on='RID', how='left')
combined

In [ ]:
import pandas as pd

# Filter for relevant categories (GS, POAG, PACG), but allow 'Other' in the mix
relevant_categories = {'GS', 'POAG', 'PACG'}
combined['Relevant'] = combined['Category'].apply(lambda x: x in relevant_categories)

# Group by 'Clinical_Records' to collect unique categories for each record
grouped = combined.groupby('Clinical_Records')['Category'].agg(set).reset_index()

# Filter to find records containing any combination of GS, POAG, and PACG
grouped['Has_Relevant_Combination'] = grouped['Category'].apply(lambda categories: len(categories.intersection(relevant_categories)) >= 2)

# Get the details of these records from the original DataFrame
detailed_records = pd.merge(grouped[grouped['Has_Relevant_Combination']], combined, on='Clinical_Records')

# Optionally, drop duplicates if you only want to see unique rows per clinical record
detailed_records = detailed_records.drop_duplicates(subset=['Clinical_Records'])

# Print the result
print(detailed_records[['Clinical_Records', 'Category_x', 'ICD10', 'Name', 'Category_y']])


In [ ]:
# Filter for rows where both 'POAG' and 'PACG' are present in the 'Category_x' column
poag_pacg_rows = detailed_records[detailed_records['Category_x'].apply(lambda x: {'POAG', 'PACG'}.issubset(x))]

# Print the filtered rows
print(poag_pacg_rows[['Clinical_Records', 'Category_x', 'ICD10', 'Name', 'Category_y']])
